# Titanic Data Preparation

## CRISP-DM

CRISP-DM is the "CRoss Industry Standard Process for Data Mining" which is one of the frequently used guidelines in data analytics. 

This notebook regards the phases "Data Understanding" and "Data Preparation". 

In [ ]:
from IPython import display
display.Image('https://upload.wikimedia.org/wikipedia/commons/b/b9/CRISP-DM_Process_Diagram.png', width=500)

## Import relevant packages

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline

## Load and inspect data set

The Kaggle Titanic data set is often used to do first steps in data science / machine learning. It contains certain personal information about passengers. The task is to prepare raw data and create prediction models, so-called classifiers, that predict if a person survived or not. Data are available here: https://www.kaggle.com/c/titanic/data - refer to this page also for further information, e.g. a data dictionary.

Attention: Reading and writing files is slightly different between Local and Cloud. 
You can click on the 0100 icon in the upper right corner, locate the file and then click "Insert to code". 
Make sure that the dataframe variable name is _original_data_ 

In [ ]:
# Local: Fetch the file
#original_data = pd.read_csv('train.csv') # Local, use full path if notebook and file in different folders! 

# Cloud: Fetch the file
# here: insert to code

original_data = pd.read_csv(body)
original_data.head()


In [ ]:
original_data.head(5)

In [ ]:
original_data.describe(include='all') # descriptive statistics for all columns

## Check for duplicates

In [ ]:
original_data[original_data.duplicated(keep=False)] # check for duplicate rows

In [ ]:
original_data['PassengerId'][original_data['PassengerId'].duplicated(keep=False)] # check for duplicate PassengerIds 

There are no obvious duplicates in the data set. 

## Check for null values and adjust data sets

In [ ]:
original_data.isnull().sum()

There are three columns that contain missing values. Simple ways to handle this data quality issues are 

a) drop the rows where a specific value is missing (here done for 'Age'), 

b) drop the whole column if too many values are missing (here: 'Cabin'), 

c) replace missing values with the most frequent value (here: 'Embarked'). 

In [ ]:
df_wo_null = original_data.dropna(axis=0, subset=['Age']) # drop rows where 'Age' is missing

In [ ]:
df_wo_null.isnull().sum()

In [ ]:
df_wo_null = df_wo_null.drop(['Cabin'], axis = 1) # drop column 'Cabin' since there are too many missing values

In [ ]:
df_wo_null.head()

In [ ]:
df_wo_null['Embarked'] = df_wo_null['Embarked'].fillna(df_wo_null['Embarked'].mode().iloc[0]) # replace missing 'Embarked' with the most frequent value

## Find predictors and edit data

In the next step, predictors are defined. These are columns (from now on called features) that are assumed to have an influence on the target, in this case 'Survived: yes or no?'. Besides, new features are derived from existing ones. This process is called 'feature engineering' and is one of the key steps during data preparation. 

### Drop features

In [ ]:
df_wo_null = df_wo_null.drop(['Ticket','PassengerId'], axis = 1) # IDs are no suitable predictors
df_wo_null = df_wo_null.drop(['Name'], axis = 1) # 'Name' is no appropriate predictor

### Survival rate per feature value

In [ ]:
df_wo_null[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).mean().sort_values(by='Survived', ascending=False)
# survival rate per class 

In [ ]:
df_wo_null[["Sex", "Survived"]].groupby(['Sex'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
viz = sns.FacetGrid(df_wo_null, col='Survived')
viz.map(plt.hist, 'Age', bins=40)

### Creating new features

In [ ]:
df_wo_null['AgeRange'] = pd.cut(df_wo_null['Age'], bins=8) # summarize 'Age' in ranges
df_wo_null[['AgeRange', 'Survived']].groupby(['AgeRange'], as_index=False).mean().sort_values(by='AgeRange', ascending=True)

In [ ]:
df_wo_null['AgeRange'] = pd.cut(df_wo_null['Age'], bins=8, labels = ['0-10','10-20','20-30','30-40','40-50','50-60','60-70', '70-80'])
# create a new column 'AgeRange'

In [ ]:
df_wo_null = df_wo_null.drop(['Age'], axis=1) # remove 'Age' since information is now contained in 'AgeRange'

In [ ]:
df_wo_null.head()

In [ ]:
viz = sns.FacetGrid(df_wo_null, col='Survived')
viz.map(plt.hist, 'Fare', bins=4)

In [ ]:
df_wo_null['FareRange'] = pd.qcut(df_wo_null['Fare'], q=4) # create quartiles from fare price 
df_wo_null[['FareRange', 'Survived']].groupby(['FareRange'], as_index=False).mean().sort_values(by='FareRange', ascending=True)

In [ ]:
df_wo_null['FareRange'] = pd.qcut(df_wo_null['Fare'], q=4, labels = ['Q1', 'Q2', 'Q3', 'Q4']) # add 'FareRange' as a new column
df_wo_null.head()

In [ ]:
df_wo_null = df_wo_null.drop(['Fare'], axis = 1) # remove 'Fare' since information is now contained in 'FareRange' 

In [ ]:
df_wo_null['Family'] = df_wo_null['SibSp'] + df_wo_null['Parch'] + 1 # calculate family size and add this as a new column

df_wo_null[['Family', 'Survived']].groupby(['Family'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
df_wo_null = df_wo_null.drop(['SibSp', 'Parch'], axis = 1) # remove columns since information is now contained in 'Family'

In [ ]:
df_wo_null.head()

In [ ]:
df_wo_null.isnull().sum()

Feel free to try out additional idea, e.g. extract title from name. 

In [ ]:
df_wo_null.head()

## Create data for modeling

In [ ]:
df_dummies = pd.get_dummies(df_wo_null, drop_first=True) # 0-1 encoding for categorical values
df_dummies.head()

In [ ]:
df_dummies.columns

Attention: Select the correct way to export csv file here. 

If you want to write a file to the project, you can follow these instructions to use ws-lib: https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/ws-lib-python.html?audience=wdp

In [2]:
# Local
# df_dummies.to_csv('train_dummies.csv', index = False) # full path if file should not be in the same folder as the notebook

# Cloud
#wslib 